# PREPROCESSING PIPELINE

**Any dataset preprocessing will happen here**

In [1]:
# Run necessary imports here

import pandas as pd
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor

# Parsing residential sales data and querying GEOID

## THIS TOOK ~115 MINUTES TO COMPLETE, DO NOT RUN UNLESS NECESSARY!

In [ ]:
fp1 = "chicago_residential_sales_data.csv"

table1 = pd.read_csv(fp1)

# property class codes for regression, check dataset documentation for more info in `DETAIL.md`
reg_class = set([200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 234, 278, 295])    
nonreg_idx = table1[~table1['Property Class'].isin(reg_class)].index

coords_label = ['Longitude', 'Latitude']

table1 = table1.drop(nonreg_idx)            # drop non-regression classes
table1 = table1.dropna(subset=coords_label) # drop NaN if lat or lon values are NaN

coords = table1[coords_label].values

url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"

params = {
    'benchmark': 4,
    'vintage':4,
    'layers': 8,
    'format': 'json'}

def fetch_census_tract(coord):
    """Fetch Census Tract GEOID for a given coordinate."""
    x, y = coord
    try:
        response = requests.get(url, params={**params, 'x': x, 'y': y}).json()
        return response['result']['geographies']['Census Tracts'][0]['GEOID']
    except Exception:
        return None  # Handle errors gracefully

# THREADING THE GEOID QUERYING, UNCOMMENT IF NEEDED
# with ThreadPoolExecutor(max_workers=20) as executor:
#     census_tracts = list(executor.map(fetch_census_tract, coords))

# table1['GEOID'] = census_tracts
# table1.to_csv("updt_chicago_residential_sales.csv")

# display(pd.read_csv("updt_chicago_residential_sales.csv"))

# Re-query missing values caused by timeouts or other exceptions during API requests and update table

In [ ]:
fp1 = "updt_chicago_residential_sales.csv"
table1 = pd.read_csv(fp1)

coords_label = ['Longitude', 'Latitude']

nulls = table1.loc[table1['GEOID'].isnull()].copy()
# display(nulls[coords_label])

url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
params = {
    'benchmark': 4,
    'vintage': 4,
    'layers': 8,
    'format': 'json'
}

def fetch_census_tract(row):
    """Fetch Census Tract GEOID for a given row's coordinates."""
    x, y = row['Longitude'], row['Latitude']
    try:
        response = requests.get(url, params={**params, 'x': x, 'y': y}).json()
        return response['result']['geographies']['Census Tracts'][0]['GEOID']
    except Exception:
        return None  # Return None if the request fails

# Use threading to speed up API calls
with ThreadPoolExecutor(max_workers=10) as executor:
    updated_geoids = list(executor.map(fetch_census_tract, nulls.to_dict(orient="records")))

# Update the GEOID column in the original table
table1.loc[table1['GEOID'].isnull(), 'GEOID'] = updated_geoids
# display(updated_geoids)

table1.to_csv("geoid_chicago_residential_sales.csv", index=False)

C:\Users\bloon\AppData\Local\Temp\ipykernel_12872\4242781246.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['17031830800', '17031823604', '17031805402', '17031740400', '17031661000', '17031661000', '17031827300', '17031730600', '17031805501', '17031660900', '17031031300', '17031820800', '17031830003', '17031842600', '17031822400', '17031821900', '17031670300', '17031818500', '17031804313', '17031801702', '17031818500', '17031661100', '17031803500', '17031050900', '17031828601', '17031200200', '17031810800', '17031830600', '17031804306', '17031828100', '17031691400', '17031420700', '17031813400', '17031801608', '17031804306', '17031805902', '17031040201', '17031803500', '17031822601', '17031560800', '17031060300', '17031830004', '17031843400', '17031825303', '17031819000', '17031150402', '17031820101', '17031819600', '17031822302', '17031825503', '17031803605', '17031832600', '17031220400', '170312

# Merging tables

In [10]:
fp1 = "geoid_chicago_residential_sales.csv"
fp2 = "chicago_data.csv"

table1 = pd.read_csv(fp1)
table2 = pd.read_csv(fp2)

data_cols = ['GEOID', 'Population', 'EKW_2024','INC_2019-2023','CZM_2023','EDB_2019-2023']

table1['GEOID'] = table1['GEOID'].astype(int)
table2['GEOID'] = table2['GEOID'].astype(int)

# print(table1['GEOID'])
# print(table2['GEOID'])

table2_sub = table2[data_cols]

merged_df = table1.merge(
    table2_sub,
    on='GEOID',
    how='left'
)

nan_rows = merged_df[merged_df[data_cols].isna().any(axis=1)]
display(nan_rows)

merged_df_clean = merged_df.dropna(subset=data_cols)
display(merged_df_clean)

merged_df_clean.to_csv("chicago_housing_data.csv", index=False)

,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,Rooms,...,Square root of lot size,Square root of age,Square root of improvement size,Town and Neighborhood,GEOID,Population,EKW_2024,INC_2019-2023,CZM_2023,EDB_2019-2023
1,4252000820000,204,100,33898.0,25,5.0,0.0,2.0,4.0,11.0,...,184.114095,7.810250,68.498175,25100,17031801902,NaN,NaN,NaN,NaN,NaN
3,27021200080000,204,34,16079.0,28,1.0,0.0,3.0,1.0,7.0,...,126.802997,5.385165,46.989360,2834,17031824105,NaN,NaN,NaN,NaN,NaN
4,1191010060000,204,21,220766.0,10,1.0,0.0,2.0,4.0,9.0,...,469.857425,5.000000,63.835727,1021,17031804204,NaN,NaN,NaN,NaN,NaN
6,7294100180000,202,40,11132.0,35,1.0,0.0,1.0,1.0,6.0,...,105.508294,7.000000,31.368774,3540,17031804807,NaN,NaN,NaN,NaN,NaN
7,16202230160000,203,30,3780.0,15,1.0,0.0,2.0,2.0,5.0,...,61.481705,9.949874,32.155870,1530,17031813400,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406618,16292300440000,203,30,4410.0,15,1.0,0.0,2.0,1.0,9.0,...,66.407831,7.681146,36.646964,1530,17031813900,NaN,NaN,NaN,NaN,NaN
406620,31111100040000,208,300,15059.0,32,2.0,0.0,2.0,1.0,9.0,...,122.715117,3.000000,67.889616,32300,17031829902,NaN,NaN,NaN,NaN,NaN
406621,31111100040000,208,300,15059.0,32,2.0,0.0,2.0,1.0,9.0,...,122.715117,3.000000,67.889616,32300,17031829902,NaN,NaN,NaN,NaN,NaN
406623,28041130320000,234,100,6050.0,13,4.0,0.0,3.0,1.0,6.0,...,77.781746,6.782330,36.715120,13100,17031824503,NaN,NaN,NaN,NaN,NaN


,PIN,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,Rooms,...,Square root of lot size,Square root of age,Square root of improvement size,Town and Neighborhood,GEOID,Population,EKW_2024,INC_2019-2023,CZM_2023,EDB_2019-2023
0,16094150130000,211,13,4500.0,77,3.0,6.0,2.0,2.0,24.0,...,67.082039,9.327379,86.226446,7713,17031251800,5101.0,14.13,36681.0,473.0,86.393773
2,14322110150000,208,12,3720.0,74,3.0,0.0,2.0,6.0,9.0,...,60.991803,2.000000,65.475186,7412,17031071100,3091.0,16.92,212677.0,124.0,100.000000
5,13121080620000,204,42,7560.0,71,1.0,0.0,2.0,1.0,6.0,...,86.948260,7.874008,50.378567,7142,17031040300,2928.0,14.89,79896.0,45.0,89.829728
9,13174070550000,203,90,4750.0,71,5.0,0.0,2.0,1.0,5.0,...,68.920244,8.602325,40.902323,7190,17031150402,3831.0,15.83,102094.0,163.0,87.607638
11,20264140050000,211,70,4687.0,70,2.0,0.0,2.0,1.0,11.0,...,68.461668,9.591663,45.716518,7070,17031834200,4693.0,14.79,63523.0,494.0,96.694215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406622,20303160470000,203,212,4000.0,72,5.0,0.0,2.0,1.0,7.0,...,63.245553,8.062258,38.884444,72212,17031710400,5424.0,12.39,43986.0,315.0,86.263591
406625,20203200240000,234,171,3125.0,72,4.0,0.0,2.0,1.0,7.0,...,55.901699,6.782330,31.128765,72171,17031834900,1952.0,12.92,20086.0,171.0,70.308483
406626,20021110150000,205,10,2750.0,70,2.0,0.0,2.0,1.0,10.0,...,52.440442,10.954451,46.281746,7010,17031836400,3621.0,13.67,27012.0,227.0,91.807591
406627,20021110150000,205,10,2750.0,70,2.0,0.0,2.0,1.0,10.0,...,52.440442,10.954451,46.281746,7010,17031836400,3621.0,13.67,27012.0,227.0,91.807591
